In [1]:
import numpy as np
import matplotlib.pyplot as plt

def objective_function(initial, test_function_no):
    x = initial

    if test_function_no == 1:
        o = 10 * -np.cos(2 * (x[0]**2 + x[1]**2)**0.5) * \
            np.exp(-0.5 * ((x[0] + 1)**2 + (x[1] - 1)**2)**0.5) + 5.1
    elif test_function_no == 2:
        o = -np.sum(np.abs(np.array([x[0], x[1]]))) + 8.5
    elif test_function_no == 3:
        o = -1 * (0.2 + x[0]**2 + x[1]**2 - 0.1 * np.cos(6 * np.pi * x[0]) -
                 0.1 * np.cos(6 * np.pi * x[1]))
    elif test_function_no == 4:
        o = -1 * np.sum(np.array(x)**2)
    else:
        raise ValueError("Invalid test_function_no")

    return o


def get_neighbours(A, step_size, ub, lb, test_function_no):
    n_var = len(A['position'])
    neighbours = []

    for idx in range(1, 5):
        if idx == 1:
            step_vector = [step_size[0], 0]
        elif idx == 2:
            step_vector = [0, step_size[1]]
        elif idx == 3:
            step_vector = [-step_size[0], 0]
        elif idx == 4:
            step_vector = [0, -step_size[1]]

        new_position = np.array(A['position']) + np.array(step_vector)

        for i in range(n_var):
            if new_position[i] > ub[i]:
                new_position[i] = ub[i]
            if new_position[i] < lb[i]:
                new_position[i] = lb[i]

        new_cost = objective_function(new_position, test_function_no)
        neighbours.append({'position': new_position, 'cost': new_cost})

    return neighbours

#### base logic of how one agent will give one solution
#### run multiple agents/iterations with different initial start point to potentially find the global optimum

In [2]:
test_function_no = 3  # choose the test function

initial = [0.8, -0.5]
cost_initial = objective_function(initial, test_function_no)

ub = [1, 1]
lb = [-1, -1]

step_size = [0.05, 0.05]


n_var = len(initial)
optimum_found = False
A = {'position': initial, 'cost': objective_function(initial, test_function_no)}
trajectory = []

iteration = 0
while not optimum_found:
    neighbours = get_neighbours(A, step_size, ub, lb, test_function_no)

    iteration += 1
    trajectory.append({'position': A['position'], 'cost': A['cost']})

    improvement = False
    for k in range(len(neighbours)):
        B = neighbours[k]
        if B['cost'] > A['cost']:
            A = {'position': B['position'], 'cost': B['cost']}
            improvement = True

    if not improvement:
        optimum_found = True

print('Best solution obtained by Hill Climbing:' , A)
for point in trajectory:
    print(point)


Best solution obtained by Hill Climbing: {'position': array([ 0.65, -0.3 ]), 'cost': -0.5364926489329899}
{'position': [0.8, -0.5], 'cost': -1.2709016994374949}
{'position': array([ 0.75, -0.5 ]), 'cost': -1.1125}
{'position': array([ 0.7, -0.5]), 'cost': -0.9590983005625051}
{'position': array([ 0.7 , -0.45]), 'cost': -0.8703768257917525}
{'position': array([ 0.7, -0.4]), 'cost': -0.7381966011250104}
{'position': array([ 0.7 , -0.35]), 'cost': -0.6364926489329898}
{'position': array([ 0.65, -0.35]), 'cost': -0.5547886967409693}
{'position': array([ 0.65, -0.3 ]), 'cost': -0.5364926489329899}
